Code for article on Medium "School scheduling with MIP"

In [2]:
from mip import Model, xsum, minimize, maximize, OptimizationStatus, BINARY
import numpy as np
from TeacherClassSchedulingData import TeacherClassSchedulingData

class TeacherClassScheduling(TeacherClassSchedulingData):

    def check(self, t, g, m, d):
        '''
        Check if the combination of Teacher, Grade, Matter and Day is valid because of reduction of variables number
        '''
        if (g,m) in self.hoursForMatterForGrade:   # Each class learn only a limited number of matters
            if m in self.teacher[t]['mattersToTeach']: # Each teacher can teach only a limited number of matters
                if g in self.teacher[t]['gradesToTeach']:  # Each teacher can teach only in a limited number of grades
                    if m in self.teacher[t]['mattersGrades'][g]: # Each teacher can teach only some matters in some grades
                        if not (d in self.desiderata['Teacher'][t]['freeDay']): # Each teacher not work in days off
                            return True
        return False
    
    def __init__(self):
        tctData = TeacherClassSchedulingData() # Let's use an external Class for handling Example Data
        self.teacher = tctData.getTeacher()
        self.grade = tctData.getGrade()
        self.matter = tctData.getMatter()
        self.day = tctData.getDay()
        self.hour = tctData.getHour()
        self.hoursForMatterForGrade = tctData.getHoursForMatterForGrade()
        self.consecutiveHours = tctData.getConsecutiveHours()
        self.desiderata =tctData.getDesiderata()
        
        self.model = Model()
        # let's build our variable x but not each combination is needed so we take care of this with the 'if' statement
        self.x = {
            (t, g, m, d, h): self.model.add_var(
                name="x({},{},{},{},{})".format(t, g, m, d, h),
                var_type=BINARY
            )
            for t in self.teacher
            for g in self.grade
            for m in self.matter
            for d in self.day
            for h in self.hour
            if self.check(t, g, m, d)
        }
        print(f"Variables x created: {len(self.x)} instead of {len(self.teacher)*len(self.grade)*len(self.matter)*len(self.day)*len(self.hour)}")    

        self.y = {
            (t): self.model.add_var(
                name="y({})".format(t),
                var_type=BINARY
            )
            for t in self.teacher
        }
        print(f"Variables y created: {len(self.y)}")    
        for t in self.teacher:
            self.y[t] = xsum( self.x[t, g, m, d, h] 
                                    for g in self.grade
                                    for m in self.matter
                                    for d in self.day
                                    for h in self.hour
                                    if self.check(t, g, m, d) ) 

        self.k = {
            (g,m): self.model.add_var(
                name="y({}{})".format(g,m),
                var_type=BINARY
            )
            for g in self.grade
            for m in self.matter
        }
        print(f"Variables k created: {len(self.k)}")    
        for g in self.grade:
            for m in self.matter:
                self.y[g,m] = xsum( self.x[t, g, m, d, h] 
                                        for m in self.teacher
                                        for d in self.day
                                        for h in self.hour
                                        if self.check(t, g, m, d) ) 
        
# ---------------------------------------------------------------------------
# Constraint Section
    def addConstraintSingle(self, t, g, m, d, h, forcing):
        '''
        Single constraint to set a particular teacher teach a particolar matter in a particular grade, day, hour. Or viceversa.
        '''
        if self.check(t, g, m, d):
            if forcing=="Force":
                self.model += self.x[t, g, m, d, h] >= 1.0
                return "Constraint of type Force built"
            if forcing=="Forbid":
                self.model += self.x[t, g, m, d, h] <= 1.0                       
                return "Constraint of type Forbid built"
            print (len(self.model.constrs))
            return "Costraint Single not built because forcing could be only Force or Forbid"
        return "Check not accomplished !"
    
    def addConstraintMinMaxHoursForTeacher(self):
        '''
        Constraint for assure that work hours for week for eaceh teacher are in range Min Max Hours for week for teacher
        '''
        for t in self.teacher:
            self.model += xsum(self.x[t, g, m, d, h] 
                          for g in self.grade 
                          for m in self.matter 
                          for d in self.day 
                          for h in self.hour 
                          if self.check(t, g, m, d) ) >= self.teacher[t]['minWeekHours']
            self.model += xsum(self.x[t, g, m, d, h] 
                          for g in self.grade 
                          for m in self.matter 
                          for d in self.day 
                          for h in self.hour 
                          if self.check(t, g, m, d) ) <= self.teacher[t]['maxWeekHours']

    def addConstraintEachHourBusyForEachGrade(self):
        '''
        Constraint for assure that each hour in each grade is busy
        '''
        for g in self.grade:
            for d in self.day:
                for h in self.hour:
                    self.model += xsum(self.x[t, g, m, d, h] 
                                  for t in self.teacher 
                                  for m in self.matter 
                                  if self.check(t, g, m, d) ) == 1.0

    def addConstraintHoursForMatterForGrade(self):
        '''
        Constraint to assure that the exact amount of hours of matter for each grade is teached
        '''
        for g in self.grade:
            for m in self.matter:
                if (g,m) in self.hoursForMatterForGrade:
                    self.model += xsum(self.x[t, g, m, d, h] 
                                  for t in self.teacher
                                  for d in self.day 
                                  for h in self.hour
                                  if self.check(t, g, m, d) ) == self.hoursForMatterForGrade[g,m]

    def addConstraintTeacherTeachInOnlyOneGradeAtSameTime(self):
        '''
        Constraint to assure that a teacher can't teach in different class at the same time, day, hour
        '''
        for t in self.teacher:
            for d in self.day:
                for h in self.hour:
                    self.model += xsum(self.x[t, g, m, d, h] 
                                  for g in self.grade 
                                  for m in self.matter 
                                  if self.check(t, g, m, d) ) <= 1.0

    def addConstraintSetGradeMatterDayHour(self):
        '''
        Constraint to assure consecutive hours for exam
        '''
        def setGradeMatterDayHour(g, m, d, h, v):
            self.model += xsum(self.x[t, g, m, d, h] 
                          for t in self.teacher 
                          if self.check(t, g, m, d) ) == v
            
        hours = list(self.hour)
        for g,m in self.consecutiveHours:
            p = hours.index(self.consecutiveHours[g,m]['hourToStart'])
            d = self.consecutiveHours[g,m]['day']
            for i in range(0,len(hours)):
                if i in range( p, p + self.consecutiveHours[g,m]['hours'] ):
                    setGradeMatterDayHour(g, m, d, hours[i], 1.0)
                else:
                    setGradeMatterDayHour(g, m, d, hours[i], 0.0)
            
    def addAllConstraint(self):
        '''
        Let's add all constraints together
        '''
        self.addConstraintSingle('Adam',
          '1A',
          'Italian',
          'Monday',
          '08:30-09:30',
          'Force')
        self.addConstraintSingle('Adam',
          '1A',
          'Italian',
          'Monday',
          '09:30-10:30',
          'Forbid')
        self.addConstraintMinMaxHoursForTeacher()
        self.addConstraintEachHourBusyForEachGrade()
        self.addConstraintHoursForMatterForGrade()
        self.addConstraintTeacherTeachInOnlyOneGradeAtSameTime()       
        self.addConstraintSetGradeMatterDayHour()
        print(f"Constraints created: {len(self.model.constrs)}")
        
# ---------------------------------------------------------------------------
    def setObjectiveFunction(self):
        pass        
    
# ---------------------------------------------------------------------------
    def optimize(self):
        self.model.optimize(max_seconds=300)
        if self.model.num_solutions:
            print(f"Feasible Solution Found ! {self.model.num_solutions}")
        else:
            print("Alert ! None feasible solution found !")
                                            
# ---------------------------------------------------------------------------
    def printTimeTableXGrade(self, grade, l=18):
        def pad(s, l=l):
            s = s[:l]
            return f"{s:<{l}}"

        def teachMat(g,d,h):
            for t in self.teacher:
                for m in self.matter:
                    if self.check(t, g, m, d):
                        if self.x[t, g, m, d, h].x == 1.0:
                            return t,m
            return "",""
        for g in grade:
            print("\r")
            print("-"*((l+1)*(len(self.day)+1)+len(self.day)))
            print (f"{pad('Grade: '+g)}"+"|", end =" ")
            for d in self.day:
                print(f"{pad(d)}"+"|", end =" ")
            print("\r")
            for h in self.hour:
                print(f"{pad(h)}|", end =" ")
                for d in self.day:
                    t,m = teachMat(g,d,h)
                    print(f"{pad(m + ' ' + t)}"+"|", end =" ")
                print("\r")
            print("-"*((l+1)*(len(self.day)+1)+len(self.day)))

    def printTimeTableXTeacher(self, teacher, l=18):
        def pad(s, l=l):
            s = s[:l]
            return f"{s:<{l}}"

        def gradeMat(t,d,h):
            for g in self.grade:
                for m in self.matter:
                    if self.check(t, g, m, d):
                        if self.x[t, g, m, d, h].x == 1.0:
                            return g,m
            return "",""
        
        for t in teacher:
            print("\r")
            print("-"*((l+1)*(len(self.day)+1)+len(self.day)))
            print (f"{pad(t)}"+"|", end =" ")
            for d in self.day:
                print(f"{pad(d)}"+"|", end =" ")
            print("\r")
            for h in self.hour:
                print(f"{pad(h)}|", end =" ")
                for d in self.day:
                    g,m = gradeMat(t,d,h)
                    print(f"{pad(g + ' ' + m)}"+"|", end =" ")
                print("\r")
            print("-"*((l+1)*(len(self.day)+1)+len(self.day)))

tct = TeacherClassScheduling()
tct.addAllConstraint()
tct.setObjectiveFunction()
tct.optimize()

tct.printTimeTableXGrade(['1A', '1B'],15)
tct.printTimeTableXTeacher(['Adam', 'John', 'Henry'], l=15)

Variables x created: 10770 instead of 28350
Variables y created: 15
Variables k created: 63
Constraints created: 902
Feasible Solution Found ! 1

----------------------------------------------------------------------------------------------------------------------
Grade: 1A      | Monday         | Tuesday        | Wednesday      | Thursday       | Friday         | Saturday       | 
08:30-09:30    | Italian Adam   | Religion George| English Frank  | Italian Adam   | Mathematic John| Italian Frank  | 
09:30-10:30    | Italian Adam   | Mathematic Char| Latin John     | Latin George   | History Frank  | Mathematic Char| 
10:30-11:30    | Italian John   | English Ida    | History Adam   | Mathematic John| English Henry  | English David  | 
11:30-12:30    | Mathematic Eve | History Adam   | Italian Adam   | English Ida    | English John   | Physical Educat| 
12:30-13:30    | Mathematic Fran| Latin Adam     | Latin Frank    | Physical Educat| Religion Ida   | History John   | 
---------------